In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore", category=DeprecationWarning) 
%matplotlib inline
sns.set()

In [2]:
hotel_data2 = pd.read_csv('hotel_data2.csv')

In [3]:
len(hotel_data2)

10000

In [4]:
hotel_data2= pd.DataFrame(hotel_data2[['id', 'address', 'city', 'country', 'latitude', 'longitude', 'name', 'postalCode', 'province', 'reviews.rating', 'reviews.text', 'reviews.title', 'sourceURLs']])
print(f"Data Shape: {hotel_data2.shape}")
hotel_data2.head()

Data Shape: (10000, 13)


,id,address,city,country,latitude,longitude,name,postalCode,province,reviews.rating,reviews.text,reviews.title,sourceURLs
0,AWE2FvX5RxPSIh2RscTK,5620 Calle Real,Goleta,US,34.44178,-119.81979,Best Western Plus South Coast Inn,93117,CA,3,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,https://www.tripadvisor.com/Hotel_Review-g3243...
1,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,4,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,http://www.tripadvisor.com/Hotel_Review-g32172...
2,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,3,"Parking was horrible, somebody ran into my ren...",Business,http://www.tripadvisor.com/Hotel_Review-g32172...
3,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,5,Not cheap but excellent location. Price is som...,Very good,http://www.tripadvisor.com/Hotel_Review-g32172...
4,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,2,If you get the room that they advertised on th...,Low chance to come back here,http://www.tripadvisor.com/Hotel_Review-g32172...


In [5]:
hotel_data2 = hotel_data2.drop_duplicates()

In [6]:
hotel_data2 = hotel_data2.rename(columns={'reviews.rating': 'reviews_rating', 'reviews.text': 'reviews_text', 'reviews.title': 'reviews_title' })

In [7]:
hotel_data2.head()

,id,address,city,country,latitude,longitude,name,postalCode,province,reviews_rating,reviews_text,reviews_title,sourceURLs
0,AWE2FvX5RxPSIh2RscTK,5620 Calle Real,Goleta,US,34.44178,-119.81979,Best Western Plus South Coast Inn,93117,CA,3,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,https://www.tripadvisor.com/Hotel_Review-g3243...
1,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,4,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,http://www.tripadvisor.com/Hotel_Review-g32172...
2,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,3,"Parking was horrible, somebody ran into my ren...",Business,http://www.tripadvisor.com/Hotel_Review-g32172...
3,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,5,Not cheap but excellent location. Price is som...,Very good,http://www.tripadvisor.com/Hotel_Review-g32172...
4,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,2,If you get the room that they advertised on th...,Low chance to come back here,http://www.tripadvisor.com/Hotel_Review-g32172...


In [8]:
hotel_data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9787 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              9787 non-null   object 
 1   address         9787 non-null   object 
 2   city            9787 non-null   object 
 3   country         9787 non-null   object 
 4   latitude        9787 non-null   float64
 5   longitude       9787 non-null   float64
 6   name            9787 non-null   object 
 7   postalCode      9787 non-null   object 
 8   province        9787 non-null   object 
 9   reviews_rating  9787 non-null   int64  
 10  reviews_text    9787 non-null   object 
 11  reviews_title   9786 non-null   object 
 12  sourceURLs      9787 non-null   object 
dtypes: float64(2), int64(1), object(10)
memory usage: 1.0+ MB


In [9]:
hotel_data2 = hotel_data2.rename(columns = {"id": "code"})

In [10]:
hotel_data2.head()

,code,address,city,country,latitude,longitude,name,postalCode,province,reviews_rating,reviews_text,reviews_title,sourceURLs
0,AWE2FvX5RxPSIh2RscTK,5620 Calle Real,Goleta,US,34.44178,-119.81979,Best Western Plus South Coast Inn,93117,CA,3,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,https://www.tripadvisor.com/Hotel_Review-g3243...
1,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,4,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,http://www.tripadvisor.com/Hotel_Review-g32172...
2,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,3,"Parking was horrible, somebody ran into my ren...",Business,http://www.tripadvisor.com/Hotel_Review-g32172...
3,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,5,Not cheap but excellent location. Price is som...,Very good,http://www.tripadvisor.com/Hotel_Review-g32172...
4,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,2,If you get the room that they advertised on th...,Low chance to come back here,http://www.tripadvisor.com/Hotel_Review-g32172...


In [11]:
hotel_data2 = hotel_data2.rename(columns = {"name": "hotel_name"})
hotel_data2.head()

,code,address,city,country,latitude,longitude,hotel_name,postalCode,province,reviews_rating,reviews_text,reviews_title,sourceURLs
0,AWE2FvX5RxPSIh2RscTK,5620 Calle Real,Goleta,US,34.44178,-119.81979,Best Western Plus South Coast Inn,93117,CA,3,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,https://www.tripadvisor.com/Hotel_Review-g3243...
1,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,4,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,http://www.tripadvisor.com/Hotel_Review-g32172...
2,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,3,"Parking was horrible, somebody ran into my ren...",Business,http://www.tripadvisor.com/Hotel_Review-g32172...
3,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,5,Not cheap but excellent location. Price is som...,Very good,http://www.tripadvisor.com/Hotel_Review-g32172...
4,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.55722,-121.92194,Best Western Carmel's Town House Lodge,93921,CA,2,If you get the room that they advertised on th...,Low chance to come back here,http://www.tripadvisor.com/Hotel_Review-g32172...


In [12]:
hotel_data2.insert(0, 'ID', range(1, 1 + len(hotel_data2)))
hotel_data2

,ID,code,address,city,country,latitude,longitude,hotel_name,postalCode,province,reviews_rating,reviews_text,reviews_title,sourceURLs
0,1,AWE2FvX5RxPSIh2RscTK,5620 Calle Real,Goleta,US,34.441780,-119.819790,Best Western Plus South Coast Inn,93117,CA,3,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,https://www.tripadvisor.com/Hotel_Review-g3243...
1,2,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,4,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,http://www.tripadvisor.com/Hotel_Review-g32172...
2,3,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,3,"Parking was horrible, somebody ran into my ren...",Business,http://www.tripadvisor.com/Hotel_Review-g32172...
3,4,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,5,Not cheap but excellent location. Price is som...,Very good,http://www.tripadvisor.com/Hotel_Review-g32172...
4,5,AVwcj_OhkufWRAb5wi9T,5th And San Carlos PO Box 3574,Carmel by the Sea,US,36.557220,-121.921940,Best Western Carmel's Town House Lodge,93921,CA,2,If you get the room that they advertised on th...,Low chance to come back here,http://www.tripadvisor.com/Hotel_Review-g32172...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9783,AVwdatg0ByjofQCxo5S5,3101 Coliseum Dr,Hampton,US,37.057650,-76.393310,Hampton Inn Hampton-newport News,23666,VA,4,My friends and I took a trip to Hampton for th...,Very accommodating and friendly staff!,https://www.tripadvisor.com/Hotel_Review-g5780...
9996,9784,AVwdatg0ByjofQCxo5S5,3101 Coliseum Dr,Hampton,US,37.057650,-76.393310,Hampton Inn Hampton-newport News,23666,VA,5,"from check in to departure, staff is friendly,...","comfortable, friendly, clean, professional",https://www.tripadvisor.com/Hotel_Review-g5780...
9997,9785,AVwdatg0ByjofQCxo5S5,3101 Coliseum Dr,Hampton,US,37.057650,-76.393310,Hampton Inn Hampton-newport News,23666,VA,5,This Hampton is located on a quiet street acro...,Great location,https://www.tripadvisor.com/Hotel_Review-g5780...
9998,9786,AV1thTgM3-Khe5l_OvT5,7886 Main Street,Hunter,US,42.210915,-74.215309,Roseberry's Inn,12442,NY,5,Awesome wings (my favorite was garlic parmesan...,Great Atmosphere!,https://www.bbb.org/us/ny/hunter/profile/hotel...


In [13]:
del hotel_data2['code']

In [14]:
del hotel_data2['sourceURLs']

In [18]:
output_path = os.path.join("Output", "hotel_review_data.csv")

#save the file
hotel_data2.to_csv(output_path,  index=False)

In [19]:
# Create variable for our connection string
conn = 'mongodb://localhost:27017'

# Pass connection string to the pymongo instance.
client = pymongo.MongoClient(conn)

# Connect to a database. 
# If the database doesn't already exist, our code will create it automatically as soon as we insert a record.
db = client.hotel_reviews

NameError: name 'pymongo' is not defined